In [1]:
# Install ffmpeg
!apt-get update
!apt-get install -y ffmpeg

# # Verify installation
# !ffmpeg -version
!pip install evaluate
!pip install jiwer
!pip install torchcodec
!pip install silero-vad

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,205 kB]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,556 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

In [2]:
!pip install --upgrade transformers accelerate

In [1]:
import pandas as pd
import os

from transformers import WhisperForConditionalGeneration
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor
import torch
from transformers import  pipeline

import json
import evaluate
from peft import PeftModel

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
ROOT_FOLDER = '/content/drive/My Drive/Project/'

checkpoint_dir = os.path.join(ROOT_FOLDER, 'finetuned_model_files','Small_R32_lr_4_10-4' , "checkpoint-4185")
base_model_name = "small"

batchsize = 8

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ROOT_FOLDER = ''
# checkpoint_dir = os.path.join(ROOT_FOLDER, 'finetuned_model_files','Small_R8_lr_3_10-4' , "checkpoint-4650")
# base_model_name = "small"

## Pretrained whisper-small Test validation res

In [4]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-" + base_model_name)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-" + base_model_name,
    language="English",      # sets English as decoding language
    task="transcribe",        # task can be 'transcribe' or 'translate'
    feature_extractor_type="whisper",  # Explicitly set feature extractor type
    feature_size=80 # Set feature size to 80
)
model.generation_config.task = 'transcribe'
model.generation_config.forced_decoder_ids = None

# Attach LoRA weights
model = PeftModel.from_pretrained(model, checkpoint_dir)

# # Read JSON file
# ROOT_FOLDER = '.'
# with open(os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json"), "r", encoding="utf-8") as f:
#     transcript_json = json.load(f)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
import sys
sys.path.append('/content/drive/My Drive/Project')
from utils.Preprocessing_inference import inference
from transformers import pipeline

In [6]:
asr_pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        torch_dtype=torch.float32,
        device=0 if torch.cuda.is_available() else -1
    )


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [7]:
inf_class = inference(os.path.join(ROOT_FOLDER, 'data','Rec' , "20241208094232_9811608043.mp3") ,asr_pipe, batchsize )

In [8]:
res = inf_class.Transcribe()

In [10]:
print(res)


 Agent :  good morning am i speaking with mister nitin jain
 Customer :  han mein bol rung apko phone aa raha hain phone bol bol ye apna ye one card se na
 Agent :  Смирщи на дали baat
 Agent :  yes yes mein gitanciali baat kar rhi hun bank of barodra card one credit card se recorded line pe
 Customer :  han ji
 Customer :  ji ji domа tahi hain
 Agent :  boliye apka kya concern hain matlab mister police
 Customer :  apka bhi hum kal bhi phone aa raha mein nahi utha paya sorry kal kan
 Agent :  ye jo call hain one card bill ke regarding aa raha tha apko matlab apka account abhi over due hain tin dinon se payment nahi huaa hain apka
 Customer :  nahi nahi mein mujhe mujhe noted hain mujhe pata hain mere saath kuchh financially problem ho gayi is wajah se delay ho raha hain mein kar nahi paya otherwise mera koi loss ho gaya maam financially koi mera paisa le ke bhaag gaya
 Agent : - 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄い問題 凄